# import

In [1]:
from model.bert import bert_ATE, bert_ABSA
from data.dataset import dataset_ATM, dataset_ABSA

In [2]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

In [4]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Acpect Term Extraction

In [4]:
laptops_train_ds = dataset_ATM(pd.read_csv("data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ATM(pd.read_csv("data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ATM(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ATM(pd.read_csv("data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ATM(pd.read_csv("data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ATM(pd.read_csv("data/twitter_test.csv"), tokenizer)

In [5]:
# w,x,y,z = laptops_train_ds.__getitem__(121)
# print(w)
# print(x)
# print(x.size())
# print(y)
# print(y.size())
# print(z)
# print(z.size())

In [8]:
train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

In [9]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [10]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [11]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [19]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ATE, 'bert_ATE.pkl')
        
def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [13]:
%time train_model_ATE(train_loader, 3)

epoch: 0  batch: 1 / 2436  loss: 0.9008495807647705  hr: 0  min: 58  sec: 5
epoch: 0  batch: 2 / 2436  loss: 0.7754075825214386  hr: 0  min: 32  sec: 27
epoch: 0  batch: 3 / 2436  loss: 0.7365630666414896  hr: 0  min: 24  sec: 25
epoch: 0  batch: 4 / 2436  loss: 0.6645084396004677  hr: 0  min: 20  sec: 16
epoch: 0  batch: 5 / 2436  loss: 0.5955630660057067  hr: 0  min: 17  sec: 27
epoch: 0  batch: 6 / 2436  loss: 0.532221553226312  hr: 0  min: 15  sec: 43
epoch: 0  batch: 7 / 2436  loss: 0.5421407371759415  hr: 0  min: 14  sec: 42
epoch: 0  batch: 8 / 2436  loss: 0.507646543905139  hr: 0  min: 13  sec: 40
epoch: 0  batch: 9 / 2436  loss: 0.5186941342221366  hr: 0  min: 12  sec: 53
epoch: 0  batch: 10 / 2436  loss: 0.49669424146413804  hr: 0  min: 12  sec: 14
epoch: 0  batch: 11 / 2436  loss: 0.47731704197146674  hr: 0  min: 11  sec: 48
epoch: 0  batch: 12 / 2436  loss: 0.45749349147081375  hr: 0  min: 11  sec: 22
epoch: 0  batch: 13 / 2436  loss: 0.4404627096194487  hr: 0  min: 11  sec

epoch: 0  batch: 109 / 2436  loss: 0.2628007786809851  hr: 0  min: 7  sec: 27
epoch: 0  batch: 110 / 2436  loss: 0.2612677312032743  hr: 0  min: 7  sec: 27
epoch: 0  batch: 111 / 2436  loss: 0.2604893575514759  hr: 0  min: 7  sec: 26
epoch: 0  batch: 112 / 2436  loss: 0.25945507089740466  hr: 0  min: 7  sec: 26
epoch: 0  batch: 113 / 2436  loss: 0.257867149547138  hr: 0  min: 7  sec: 25
epoch: 0  batch: 114 / 2436  loss: 0.2567254625130118  hr: 0  min: 7  sec: 25
epoch: 0  batch: 115 / 2436  loss: 0.25588474286639173  hr: 0  min: 7  sec: 25
epoch: 0  batch: 116 / 2436  loss: 0.25498196868033246  hr: 0  min: 7  sec: 25
epoch: 0  batch: 117 / 2436  loss: 0.25420687022881633  hr: 0  min: 7  sec: 25
epoch: 0  batch: 118 / 2436  loss: 0.25289619303608346  hr: 0  min: 7  sec: 24
epoch: 0  batch: 119 / 2436  loss: 0.252523228087846  hr: 0  min: 7  sec: 25
epoch: 0  batch: 120 / 2436  loss: 0.2512171743437648  hr: 0  min: 7  sec: 25
epoch: 0  batch: 121 / 2436  loss: 0.25001167887744824  hr: 0

epoch: 0  batch: 214 / 2436  loss: 0.19984295511635664  hr: 0  min: 7  sec: 8
epoch: 0  batch: 215 / 2436  loss: 0.19942043142956356  hr: 0  min: 7  sec: 8
epoch: 0  batch: 216 / 2436  loss: 0.19902576119811446  hr: 0  min: 7  sec: 7
epoch: 0  batch: 217 / 2436  loss: 0.19884479938564212  hr: 0  min: 7  sec: 7
epoch: 0  batch: 218 / 2436  loss: 0.19852806443194732  hr: 0  min: 7  sec: 7
epoch: 0  batch: 219 / 2436  loss: 0.19809431327532415  hr: 0  min: 7  sec: 7
epoch: 0  batch: 220 / 2436  loss: 0.19769290001554923  hr: 0  min: 7  sec: 8
epoch: 0  batch: 221 / 2436  loss: 0.1975522286751691  hr: 0  min: 7  sec: 9
epoch: 0  batch: 222 / 2436  loss: 0.19722146020682008  hr: 0  min: 7  sec: 8
epoch: 0  batch: 223 / 2436  loss: 0.19687118488175986  hr: 0  min: 7  sec: 8
epoch: 0  batch: 224 / 2436  loss: 0.1966379729565233  hr: 0  min: 7  sec: 8
epoch: 0  batch: 225 / 2436  loss: 0.19639723774459628  hr: 0  min: 7  sec: 8
epoch: 0  batch: 226 / 2436  loss: 0.19605761238958982  hr: 0  min

epoch: 0  batch: 321 / 2436  loss: 0.16647888652725013  hr: 0  min: 6  sec: 57
epoch: 0  batch: 322 / 2436  loss: 0.16628044899205985  hr: 0  min: 6  sec: 57
epoch: 0  batch: 323 / 2436  loss: 0.16603747875236505  hr: 0  min: 6  sec: 57
epoch: 0  batch: 324 / 2436  loss: 0.1657918598042962  hr: 0  min: 6  sec: 57
epoch: 0  batch: 325 / 2436  loss: 0.16551379178579037  hr: 0  min: 6  sec: 56
epoch: 0  batch: 326 / 2436  loss: 0.16537281338331158  hr: 0  min: 6  sec: 57
epoch: 0  batch: 327 / 2436  loss: 0.16504726170445436  hr: 0  min: 6  sec: 57
epoch: 0  batch: 328 / 2436  loss: 0.16496319025073472  hr: 0  min: 6  sec: 57
epoch: 0  batch: 329 / 2436  loss: 0.16466782732348673  hr: 0  min: 6  sec: 57
epoch: 0  batch: 330 / 2436  loss: 0.16429630149946067  hr: 0  min: 6  sec: 57
epoch: 0  batch: 331 / 2436  loss: 0.1641859852250249  hr: 0  min: 6  sec: 56
epoch: 0  batch: 332 / 2436  loss: 0.16417302575007262  hr: 0  min: 6  sec: 56
epoch: 0  batch: 333 / 2436  loss: 0.16385538598617633

epoch: 0  batch: 427 / 2436  loss: 0.1450565709390043  hr: 0  min: 6  sec: 49
epoch: 0  batch: 428 / 2436  loss: 0.14492680700756957  hr: 0  min: 6  sec: 49
epoch: 0  batch: 429 / 2436  loss: 0.1449531424174081  hr: 0  min: 6  sec: 49
epoch: 0  batch: 430 / 2436  loss: 0.14499399898530438  hr: 0  min: 6  sec: 49
epoch: 0  batch: 431 / 2436  loss: 0.14474672506068534  hr: 0  min: 6  sec: 49
epoch: 0  batch: 432 / 2436  loss: 0.14453024503395515  hr: 0  min: 6  sec: 49
epoch: 0  batch: 433 / 2436  loss: 0.14436815761666374  hr: 0  min: 6  sec: 49
epoch: 0  batch: 434 / 2436  loss: 0.14416576895330632  hr: 0  min: 6  sec: 48
epoch: 0  batch: 435 / 2436  loss: 0.1439383768315973  hr: 0  min: 6  sec: 48
epoch: 0  batch: 436 / 2436  loss: 0.1438143388496353  hr: 0  min: 6  sec: 48
epoch: 0  batch: 437 / 2436  loss: 0.1437248313965732  hr: 0  min: 6  sec: 48
epoch: 0  batch: 438 / 2436  loss: 0.1435522870692216  hr: 0  min: 6  sec: 48
epoch: 0  batch: 439 / 2436  loss: 0.14330145246550813  hr

...

epoch: 2  batch: 2004 / 2436  loss: 0.008051921706124947  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2005 / 2436  loss: 0.008050054516493411  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2006 / 2436  loss: 0.00804730760901003  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2007 / 2436  loss: 0.008043711015182616  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2008 / 2436  loss: 0.008065339922210346  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2009 / 2436  loss: 0.008063316919539244  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2010 / 2436  loss: 0.008060266124850196  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2011 / 2436  loss: 0.008056467894835382  hr: 0  min: 0  sec: 25
epoch: 2  batch: 2012 / 2436  loss: 0.008055303047276141  hr: 0  min: 0  sec: 24
epoch: 2  batch: 2013 / 2436  loss: 0.008053986898426013  hr: 0  min: 0  sec: 24
epoch: 2  batch: 2014 / 2436  loss: 0.008055637850540052  hr: 0  min: 0  sec: 24
epoch: 2  batch: 2015 / 2436  loss: 0.008051954861800749  hr: 0  min: 0  sec: 24
epoch: 2  batch: 2016 / 2436 

epoch: 2  batch: 2109 / 2436  loss: 0.008013151309360644  hr: 0  min: 0  sec: 19
epoch: 2  batch: 2110 / 2436  loss: 0.008009601911489278  hr: 0  min: 0  sec: 19
epoch: 2  batch: 2111 / 2436  loss: 0.008006022011504391  hr: 0  min: 0  sec: 19
epoch: 2  batch: 2112 / 2436  loss: 0.008002456373689753  hr: 0  min: 0  sec: 19
epoch: 2  batch: 2113 / 2436  loss: 0.008019012975920673  hr: 0  min: 0  sec: 19
epoch: 2  batch: 2114 / 2436  loss: 0.008016260432724716  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2115 / 2436  loss: 0.008012638313700398  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2116 / 2436  loss: 0.008013308556593936  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2117 / 2436  loss: 0.008009761502889724  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2118 / 2436  loss: 0.00800608960313361  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2119 / 2436  loss: 0.008002369084598178  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2120 / 2436  loss: 0.007998785936706554  hr: 0  min: 0  sec: 18
epoch: 2  batch: 2121 / 2436 

epoch: 2  batch: 2212 / 2436  loss: 0.008050591705090693  hr: 0  min: 0  sec: 13
epoch: 2  batch: 2213 / 2436  loss: 0.008047373570452276  hr: 0  min: 0  sec: 13
epoch: 2  batch: 2214 / 2436  loss: 0.0080439944451038  hr: 0  min: 0  sec: 13
epoch: 2  batch: 2215 / 2436  loss: 0.008040570495621922  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2216 / 2436  loss: 0.008037618919695474  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2217 / 2436  loss: 0.008034719865861488  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2218 / 2436  loss: 0.008041086818310677  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2219 / 2436  loss: 0.008037860098934254  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2220 / 2436  loss: 0.008034680269348256  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2221 / 2436  loss: 0.008031203232038263  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2222 / 2436  loss: 0.008027827416765878  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2223 / 2436  loss: 0.008034485824329505  hr: 0  min: 0  sec: 12
epoch: 2  batch: 2224 / 2436  

epoch: 2  batch: 2315 / 2436  loss: 0.007900216871826406  hr: 0  min: 0  sec: 7
epoch: 2  batch: 2316 / 2436  loss: 0.007897305105660567  hr: 0  min: 0  sec: 7
epoch: 2  batch: 2317 / 2436  loss: 0.007894705744596211  hr: 0  min: 0  sec: 7
epoch: 2  batch: 2318 / 2436  loss: 0.007897035898914087  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2319 / 2436  loss: 0.007893752236898878  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2320 / 2436  loss: 0.00789037974284305  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2321 / 2436  loss: 0.007900720642026704  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2322 / 2436  loss: 0.007901945968060051  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2323 / 2436  loss: 0.007898748881663835  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2324 / 2436  loss: 0.007895494294540551  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2325 / 2436  loss: 0.007899603181407831  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2326 / 2436  loss: 0.007896332474090878  hr: 0  min: 0  sec: 6
epoch: 2  batch: 2327 / 2436  loss: 0.007

epoch: 2  batch: 2421 / 2436  loss: 0.007778141335161911  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2422 / 2436  loss: 0.007774969676254294  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2423 / 2436  loss: 0.007771895489332879  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2424 / 2436  loss: 0.007771642556975649  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2425 / 2436  loss: 0.0077906617102694465  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2426 / 2436  loss: 0.007788709656496576  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2427 / 2436  loss: 0.0077855538082609865  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2428 / 2436  loss: 0.0077847603025374724  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2429 / 2436  loss: 0.007781653193422861  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2430 / 2436  loss: 0.007789727940854455  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2431 / 2436  loss: 0.007786557909014882  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2432 / 2436  loss: 0.007817474972102301  hr: 0  min: 0  sec: 0
epoch: 2  batch: 2433 / 2436  loss: 0

In [14]:
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

In [18]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

Wall time: 23.1 s
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    140373
           1       0.84      0.92      0.88      6486
           2       0.93      0.73      0.82      3837

    accuracy                           0.98    150696
   macro avg       0.92      0.88      0.90    150696
weighted avg       0.99      0.98      0.98    150696



# Aspect Based Sentiment Analysis

In [5]:
laptops_train_ds = dataset_ABSA(pd.read_csv("data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ABSA(pd.read_csv("data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ABSA(pd.read_csv("data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ABSA(pd.read_csv("data/twitter_test.csv"), tokenizer)

In [6]:
w,x,y,z = laptops_train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'the', 'battery', 'life', 'seems', 'to', 'be', 'very', 'good', ',', 'and', 'have', 'had', 'no', 'issues', 'with', 'it', '.', '[sep]', 'battery', 'life']
21
tensor([ 100, 1996, 6046, 2166, 3849, 2000, 2022, 2200, 2204, 1010, 1998, 2031,
        2018, 2053, 3314, 2007, 2009, 1012,  100, 6046, 2166])
21
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
21
tensor(2)


In [7]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [8]:
train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

In [9]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [20]:
def train_model_ABSA(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ABSA, 'bert_ABSA2.pkl')
        
def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [55]:
%time train_model_ABSA(train_loader, 6)

epoch: 0  batch: 1 / 3044  loss: 0.013607997447252274  hr: 0  min: 22  sec: 31
epoch: 0  batch: 2 / 3044  loss: 0.015948042273521423  hr: 0  min: 22  sec: 3
epoch: 0  batch: 3 / 3044  loss: 0.012159172290315231  hr: 0  min: 22  sec: 13
epoch: 0  batch: 4 / 3044  loss: 0.010444243671372533  hr: 0  min: 22  sec: 31
epoch: 0  batch: 5 / 3044  loss: 0.00979887628927827  hr: 0  min: 21  sec: 25
epoch: 0  batch: 6 / 3044  loss: 0.009676736546680331  hr: 0  min: 21  sec: 5
epoch: 0  batch: 7 / 3044  loss: 0.008744179071592433  hr: 0  min: 20  sec: 41
epoch: 0  batch: 8 / 3044  loss: 0.008237812086008489  hr: 0  min: 20  sec: 20
epoch: 0  batch: 9 / 3044  loss: 0.012049233644372888  hr: 0  min: 19  sec: 58
epoch: 0  batch: 10 / 3044  loss: 0.011414130451157689  hr: 0  min: 20  sec: 1
epoch: 0  batch: 11 / 3044  loss: 0.010607798329808495  hr: 0  min: 19  sec: 51
epoch: 0  batch: 12 / 3044  loss: 0.0104247546599557  hr: 0  min: 19  sec: 38
epoch: 0  batch: 13 / 3044  loss: 0.00978194370579261  

epoch: 0  batch: 104 / 3044  loss: 0.016129425748658832  hr: 0  min: 17  sec: 36
epoch: 0  batch: 105 / 3044  loss: 0.0160016957753604  hr: 0  min: 17  sec: 36
epoch: 0  batch: 106 / 3044  loss: 0.015879426252004638  hr: 0  min: 17  sec: 35
epoch: 0  batch: 107 / 3044  loss: 0.015754646788651523  hr: 0  min: 17  sec: 35
epoch: 0  batch: 108 / 3044  loss: 0.015615760819249166  hr: 0  min: 17  sec: 35
epoch: 0  batch: 109 / 3044  loss: 0.015488056512442717  hr: 0  min: 17  sec: 34
epoch: 0  batch: 110 / 3044  loss: 0.015736480662070045  hr: 0  min: 17  sec: 35
epoch: 0  batch: 111 / 3044  loss: 0.01570194261935162  hr: 0  min: 17  sec: 35
epoch: 0  batch: 112 / 3044  loss: 0.015616736461587217  hr: 0  min: 17  sec: 36
epoch: 0  batch: 113 / 3044  loss: 0.015540936421417524  hr: 0  min: 17  sec: 36
epoch: 0  batch: 114 / 3044  loss: 0.01541880232418822  hr: 0  min: 17  sec: 35
epoch: 0  batch: 115 / 3044  loss: 0.01568471813807264  hr: 0  min: 17  sec: 35
epoch: 0  batch: 116 / 3044  loss

epoch: 0  batch: 209 / 3044  loss: 0.024807987404089314  hr: 0  min: 17  sec: 23
epoch: 0  batch: 210 / 3044  loss: 0.024702997706619847  hr: 0  min: 17  sec: 23
epoch: 0  batch: 211 / 3044  loss: 0.02459154933083017  hr: 0  min: 17  sec: 24
epoch: 0  batch: 212 / 3044  loss: 0.02448006660539112  hr: 0  min: 17  sec: 23
epoch: 0  batch: 213 / 3044  loss: 0.02438189963943504  hr: 0  min: 17  sec: 23
epoch: 0  batch: 214 / 3044  loss: 0.02427201894836535  hr: 0  min: 17  sec: 23
epoch: 0  batch: 215 / 3044  loss: 0.024164378358765917  hr: 0  min: 17  sec: 23
epoch: 0  batch: 216 / 3044  loss: 0.02405899948455583  hr: 0  min: 17  sec: 22
epoch: 0  batch: 217 / 3044  loss: 0.023951578502272888  hr: 0  min: 17  sec: 22
epoch: 0  batch: 218 / 3044  loss: 0.023896787352988138  hr: 0  min: 17  sec: 22
epoch: 0  batch: 219 / 3044  loss: 0.023810914501794466  hr: 0  min: 17  sec: 22
epoch: 0  batch: 220 / 3044  loss: 0.023708966671256347  hr: 0  min: 17  sec: 22
epoch: 0  batch: 221 / 3044  loss

epoch: 0  batch: 311 / 3044  loss: 0.021119479589700292  hr: 0  min: 17  sec: 14
epoch: 0  batch: 312 / 3044  loss: 0.021054819827930678  hr: 0  min: 17  sec: 14
epoch: 0  batch: 313 / 3044  loss: 0.020994400720300968  hr: 0  min: 17  sec: 14
epoch: 0  batch: 314 / 3044  loss: 0.02092940702398195  hr: 0  min: 17  sec: 14
epoch: 0  batch: 315 / 3044  loss: 0.02086572757800154  hr: 0  min: 17  sec: 14
epoch: 0  batch: 316 / 3044  loss: 0.020802163644038033  hr: 0  min: 17  sec: 14
epoch: 0  batch: 317 / 3044  loss: 0.020791841613397602  hr: 0  min: 17  sec: 14
epoch: 0  batch: 318 / 3044  loss: 0.020737390396142752  hr: 0  min: 17  sec: 14
epoch: 0  batch: 319 / 3044  loss: 0.02067477059120686  hr: 0  min: 17  sec: 14
epoch: 0  batch: 320 / 3044  loss: 0.020612160923883495  hr: 0  min: 17  sec: 13
epoch: 0  batch: 321 / 3044  loss: 0.020574415428817817  hr: 0  min: 17  sec: 13
epoch: 0  batch: 322 / 3044  loss: 0.02051929941293147  hr: 0  min: 17  sec: 13
epoch: 0  batch: 323 / 3044  los

epoch: 0  batch: 416 / 3044  loss: 0.026728903328988123  hr: 0  min: 17  sec: 8
epoch: 0  batch: 417 / 3044  loss: 0.026783594804047104  hr: 0  min: 17  sec: 8
epoch: 0  batch: 418 / 3044  loss: 0.02672077593529813  hr: 0  min: 17  sec: 8
epoch: 0  batch: 419 / 3044  loss: 0.02666274114209894  hr: 0  min: 17  sec: 7
epoch: 0  batch: 420 / 3044  loss: 0.026605825442224854  hr: 0  min: 17  sec: 8
epoch: 0  batch: 421 / 3044  loss: 0.026547495045739552  hr: 0  min: 17  sec: 7
epoch: 0  batch: 422 / 3044  loss: 0.026567931017814694  hr: 0  min: 17  sec: 7
epoch: 0  batch: 423 / 3044  loss: 0.02652070233627249  hr: 0  min: 17  sec: 7
epoch: 0  batch: 424 / 3044  loss: 0.026462960170149773  hr: 0  min: 17  sec: 7
epoch: 0  batch: 425 / 3044  loss: 0.026403115817195025  hr: 0  min: 17  sec: 7
epoch: 0  batch: 426 / 3044  loss: 0.026365962481039288  hr: 0  min: 17  sec: 7
epoch: 0  batch: 427 / 3044  loss: 0.02630546603214696  hr: 0  min: 17  sec: 7
epoch: 0  batch: 428 / 3044  loss: 0.0262461

...

epoch: 5  batch: 2533 / 3044  loss: 0.036638872824021965  hr: 0  min: 0  sec: 30
epoch: 5  batch: 2534 / 3044  loss: 0.0367775292866024  hr: 0  min: 0  sec: 30
epoch: 5  batch: 2535 / 3044  loss: 0.03676334907132806  hr: 0  min: 0  sec: 30
epoch: 5  batch: 2536 / 3044  loss: 0.0367493633453269  hr: 0  min: 0  sec: 30
epoch: 5  batch: 2537 / 3044  loss: 0.036855675618034704  hr: 0  min: 0  sec: 30
epoch: 5  batch: 2538 / 3044  loss: 0.036841555084280976  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2539 / 3044  loss: 0.03682739959433591  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2540 / 3044  loss: 0.03681348157686472  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2541 / 3044  loss: 0.036799084171536885  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2542 / 3044  loss: 0.03678710585788475  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2543 / 3044  loss: 0.036772929935847846  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2544 / 3044  loss: 0.03675857483594081  hr: 0  min: 0  sec: 29
epoch: 5  batch: 2545 / 3044  loss: 0

epoch: 5  batch: 2638 / 3044  loss: 0.0358993664570316  hr: 0  min: 0  sec: 24
epoch: 5  batch: 2639 / 3044  loss: 0.03588578512529995  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2640 / 3044  loss: 0.035872479578038664  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2641 / 3044  loss: 0.0358593663409353  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2642 / 3044  loss: 0.03592219824705965  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2643 / 3044  loss: 0.03590882407087428  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2644 / 3044  loss: 0.035895818629679895  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2645 / 3044  loss: 0.035884335930858345  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2646 / 3044  loss: 0.03587161069106154  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2647 / 3044  loss: 0.03585825296105788  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2648 / 3044  loss: 0.03584600657128589  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2649 / 3044  loss: 0.035833778120256905  hr: 0  min: 0  sec: 23
epoch: 5  batch: 2650 / 3044  loss: 0.

epoch: 5  batch: 2741 / 3044  loss: 0.03542057629413445  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2742 / 3044  loss: 0.035407952852777015  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2743 / 3044  loss: 0.035395319707592264  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2744 / 3044  loss: 0.035383041432848664  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2745 / 3044  loss: 0.03537020438771205  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2746 / 3044  loss: 0.03535766011833444  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2747 / 3044  loss: 0.035345336733316406  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2748 / 3044  loss: 0.03554532727069801  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2749 / 3044  loss: 0.035532908472570617  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2750 / 3044  loss: 0.03552301214171166  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2751 / 3044  loss: 0.03551026852347984  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2752 / 3044  loss: 0.03549773063693417  hr: 0  min: 0  sec: 17
epoch: 5  batch: 2753 / 3044  loss:

epoch: 5  batch: 2843 / 3044  loss: 0.0355331560544469  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2844 / 3044  loss: 0.035520951425226696  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2845 / 3044  loss: 0.03550980617882869  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2846 / 3044  loss: 0.035498090715126274  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2847 / 3044  loss: 0.035486689554561086  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2848 / 3044  loss: 0.03547457803450583  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2849 / 3044  loss: 0.035462431237883806  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2850 / 3044  loss: 0.03545051519588844  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2851 / 3044  loss: 0.03543840413950876  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2852 / 3044  loss: 0.035426141072694546  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2853 / 3044  loss: 0.03541386182756323  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2854 / 3044  loss: 0.035401672192353933  hr: 0  min: 0  sec: 11
epoch: 5  batch: 2855 / 3044  loss:

epoch: 5  batch: 2948 / 3044  loss: 0.035328189131411315  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2949 / 3044  loss: 0.03531657375205926  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2950 / 3044  loss: 0.0353053589764836  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2951 / 3044  loss: 0.03529410910093402  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2952 / 3044  loss: 0.03528279279223927  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2953 / 3044  loss: 0.03527091240225668  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2954 / 3044  loss: 0.03525965061833024  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2955 / 3044  loss: 0.03534811287155006  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2956 / 3044  loss: 0.03533775101338736  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2957 / 3044  loss: 0.03532583492772173  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2958 / 3044  loss: 0.03531471698885935  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2959 / 3044  loss: 0.03530308177872621  hr: 0  min: 0  sec: 5
epoch: 5  batch: 2960 / 3044  loss: 0.03529139945558

In [12]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA.pkl')

In [13]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

Wall time: 10.1 s
              precision    recall  f1-score   support

           0       0.72      0.75      0.74       497
           1       0.67      0.74      0.70       710
           2       0.89      0.83      0.86      1239

    accuracy                           0.79      2446
   macro avg       0.76      0.77      0.77      2446
weighted avg       0.79      0.79      0.79      2446



# ATE + ABSA

In [57]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def ATE_ABSA(text):
    terms = []
    word = ""
    x, y, z = predict_model_ATE(text, tokenizer)
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##",""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])
            
    
    if len(word) != 0:
            terms.append(word.replace(" ##",""))
            
    print("tokens:", x)
    print("ATE:", terms)
    
    if len(terms) != 0:
        for i in terms:
            _, c, p = predict_model_ABSA(text, i, tokenizer)
            print("term:", [i], "class:", [int(c)], "ABSA:", [float(p[0][0]), float(p[0][1]), float(p[0][2])])


In [58]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA.pkl')
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

In [59]:
text = "For the price you pay this product is very good. However, battery life is a little lack-luster coming from a MacBook Pro."
ATE_ABSA(text)

tokens: ['for', 'the', 'price', 'you', 'pay', 'this', 'product', 'is', 'very', 'good', '.', 'however', ',', 'battery', 'life', 'is', 'a', 'little', 'lack', '-', 'lust', '##er', 'coming', 'from', 'a', 'mac', '##book', 'pro', '.']
ATE: ['price', 'battery life']
term: ['price'] class: [2] ABSA: [-2.057527542114258, -0.6292028427124023, 2.606888771057129]
term: ['battery life'] class: [0] ABSA: [5.0118207931518555, -2.3663508892059326, -1.8548927307128906]


In [60]:
text = "I think Apple is better than Microsoft."
ATE_ABSA(text)

tokens: ['i', 'think', 'apple', 'is', 'better', 'than', 'microsoft', '.']
ATE: ['apple', 'microsoft']
term: ['apple'] class: [1] ABSA: [-0.6019173264503479, 1.7071634531021118, -0.6558032631874084]
term: ['microsoft'] class: [0] ABSA: [3.7963366508483887, 0.2745559811592102, -3.170161485671997]


# Cyberpunk 2077 - Xbox One

https://www.amazon.com/-/zh_TW/Cyberpunk-2077-Xbox-One/product-reviews/B07DJW4WZC/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2

In [61]:
text = "Spent 5 hours downloading updates."
ATE_ABSA(text)

tokens: ['spent', '5', 'hours', 'download', '##ing', 'updates', '.']
ATE: ['download', '##ing updates']
term: ['download'] class: [1] ABSA: [-1.8432576656341553, 5.523550987243652, -3.1730458736419678]
term: ['##ing updates'] class: [1] ABSA: [-2.6808316707611084, 6.011416912078857, -2.864603042602539]


In [62]:
text = "Install is buggy, so after downloading a day one patch that's nearly 3 times the size of the game, it glitched on the CDs and had to reinstall the game from scratch."
ATE_ABSA(text)

tokens: ['install', 'is', 'bug', '##gy', ',', 'so', 'after', 'download', '##ing', 'a', 'day', 'one', 'patch', 'that', "'", 's', 'nearly', '3', 'times', 'the', 'size', 'of', 'the', 'game', ',', 'it', 'g', '##lit', '##ched', 'on', 'the', 'cds', 'and', 'had', 'to', 'reins', '##tal', '##l', 'the', 'game', 'from', 'scratch', '.']
ATE: ['install', 'patch', 'size', 'game', 'cds', 'game']
term: ['install'] class: [1] ABSA: [-2.2875146865844727, 4.987157821655273, -2.462806463241577]
term: ['patch'] class: [0] ABSA: [3.8425865173339844, -1.115385890007019, -1.8236310482025146]
term: ['size'] class: [1] ABSA: [1.2320547103881836, 1.6156060695648193, -2.0953636169433594]
term: ['game'] class: [1] ABSA: [-2.642533779144287, 5.77440071105957, -2.5892865657806396]
term: ['cds'] class: [1] ABSA: [-2.673746347427368, 5.815953731536865, -2.6103947162628174]
term: ['game'] class: [1] ABSA: [-2.642533779144287, 5.77440071105957, -2.5892865657806396]


In [71]:
text = "Cyberpunk 2077 freezes constantly, frame rates are terrible, and it's extremely frustrating to try to play."
ATE_ABSA(text)

tokens: ['cyber', '##pu', '##nk', '207', '##7', 'freeze', '##s', 'constantly', ',', 'frame', 'rates', 'are', 'terrible', ',', 'and', 'it', "'", 's', 'extremely', 'frustrating', 'to', 'try', 'to', 'play', '.']
ATE: ['cyberpu', 'frame rates']
term: ['cyberpu'] class: [0] ABSA: [4.44415283203125, -0.36560752987861633, -3.3459084033966064]
term: ['frame rates'] class: [0] ABSA: [5.2562408447265625, -2.305537700653076, -2.0652124881744385]


In [66]:
text = "Cyberpunk 2077 is completely unplayable on xbox one. They should have never released this for current gen."
ATE_ABSA(text)

tokens: ['cyber', '##pu', '##nk', '207', '##7', 'is', 'completely', 'un', '##play', '##able', 'on', 'xbox', 'one', '.', 'they', 'should', 'have', 'never', 'released', 'this', 'for', 'current', 'gen', '.']
ATE: ['xbox']
term: ['xbox'] class: [1] ABSA: [-2.0123980045318604, 5.7579731941223145, -3.235884666442871]


In [67]:
text = "It’s just a cash grab, the game crashes constantly, runs at like 20 fps, half the environment and characters only load when you’re three feet away from them. Unless you’re in a small space the game looks awful. The worst game i’ve ever played in years visually. It looks worse than later xbox 360 games."
ATE_ABSA(text)

tokens: ['it', '’', 's', 'just', 'a', 'cash', 'grab', ',', 'the', 'game', 'crashes', 'constantly', ',', 'runs', 'at', 'like', '20', 'f', '##ps', ',', 'half', 'the', 'environment', 'and', 'characters', 'only', 'load', 'when', 'you', '’', 're', 'three', 'feet', 'away', 'from', 'them', '.', 'unless', 'you', '’', 're', 'in', 'a', 'small', 'space', 'the', 'game', 'looks', 'awful', '.', 'the', 'worst', 'game', 'i', '’', 've', 'ever', 'played', 'in', 'years', 'visually', '.', 'it', 'looks', 'worse', 'than', 'later', 'xbox', '360', 'games', '.']
ATE: ['runs', 'environment', 'characters', 'xbox 360']
term: ['runs'] class: [0] ABSA: [4.292036056518555, -0.47526031732559204, -3.092445135116577]
term: ['environment'] class: [0] ABSA: [3.7919883728027344, 0.2691774070262909, -3.255924940109253]
term: ['characters'] class: [0] ABSA: [3.6958675384521484, 0.34112149477005005, -3.180785655975342]
term: ['xbox 360'] class: [0] ABSA: [3.6143949031829834, 0.5955154299736023, -3.378544330596924]


In [68]:
text = "CD Projekt Red should have just abandoned the current gen consoles instead of cheating people out of their money."
ATE_ABSA(text)

tokens: ['cd', 'pro', '##je', '##kt', 'red', 'should', 'have', 'just', 'abandoned', 'the', 'current', 'gen', 'consoles', 'instead', 'of', 'cheating', 'people', 'out', 'of', 'their', 'money', '.']
ATE: ['gen', 'consoles']
term: ['gen'] class: [1] ABSA: [-2.468400478363037, 5.807504177093506, -2.8556580543518066]
term: ['consoles'] class: [1] ABSA: [-2.6495633125305176, 5.937881946563721, -2.8111135959625244]
